# 动态路由

链接：https://python.langchain.com/docs/expression_language/how_to/routing/

# 初始化

In [ ]:
import os

API_KEY = os.getenv("UNION_API_KEY")
BASE_URL = os.getenv("UNION_BASE_URL")

# 测试Claude AI的接口

In [3]:
from openai import OpenAI

# api_key请替换为网站生成的API令牌
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# 创建一个请求来获取Claude-3的回应
completion = client.chat.completions.create(
    model="claude-3-haiku-20240307",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello"}
    ]
)

# 打印回应内容
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0, api_key=API_KEY, model_name="claude-3-haiku-20240307", base_url=BASE_URL)

# chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

print(prompt)

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

input_variables=['input_language', 'output_language', 'text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], template='You are a helpful assistant that translates {input_language} to {output_language}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]


AIMessage(content='파이썬(Python)을 사랑합니다.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 22, 'total_tokens': 42, 'total_cost': 3.05e-05}, 'model_name': 'claude-3-haiku-20240307', 'system_fingerprint': None, 'finish_reason': 'end_turn', 'logprobs': None}, id='run-403f9305-80c8-44f6-b7e9-8f2a8c4d5cdc-0')

In [13]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0, api_key=API_KEY, model_name="gpt-3.5-turbo", base_url=BASE_URL)

# chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

AIMessage(content='나는 파이썬을 좋아해요.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 25, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-c58a31a4-17bd-497b-86ca-b486a33e002b-0')

# 根据输入动态路由

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# 定义一个模型，和Claude-3对话
chat_anthropic = ChatOpenAI(temperature=0, api_key=API_KEY, model_name="claude-3-haiku-20240307", base_url=BASE_URL)

chain =(
    PromptTemplate.from_template(
"""Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
)
| chat_anthropic
| StrOutputParser()
)

chain.invoke({"question":"how do I call Anthropic?"})

################# 创建3个子链 ##########################
# 子链1：LangChain专家
langchain_chain =(
    PromptTemplate.from_template(
"""You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
)
| chat_anthropic
)

# 子链2：Anthropic专家
anthropic_chain =(
    PromptTemplate.from_template(
"""You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
)
| chat_anthropic
)

# 子链3：全能专家
general_chain =(
    PromptTemplate.from_template(
"""Respond to the following question:

Question: {question}
Answer:"""
)
| chat_anthropic
)


from langchain_core.runnables import RunnableBranch

# 创建一个RunnableBranch，根据问题的主题，选择对应的子链
branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

# 创建一个完整的链，包括问题分类和子链选择
# 首先通过对 chain 的调用进行分类，然后根据 question 选择对应的子链来进行后继处理
full_chain ={"topic": chain,"question":lambda x: x["question"]}| branch

QUESTIONS = [
    "how do I use Anthropic?",
    "how do I use Langchain?",
    "how do I read a book?"
]

for question in QUESTIONS:
    print("-" * 50)
    result = full_chain.invoke({"question": question})
    print(result)

--------------------------------------------------
content="As Dario Amodei told me, to use Anthropic, you can start by exploring the various AI models and tools they offer. Anthropic provides a range of language models, including GPT-3 and other advanced AI systems, that can be used for a variety of tasks such as natural language processing, text generation, and question answering. \n\nTo get started, you can visit the Anthropic website and learn about their products and services. They offer APIs and SDKs that allow you to integrate Anthropic's AI capabilities into your own applications and workflows. Additionally, Anthropic has a strong focus on responsible AI development, so they provide guidance and resources on ethical AI practices.\n\nI'd recommend reviewing the Anthropic documentation and exploring their developer tools to understand how you can leverage their AI technology to meet your specific needs. Dario and the Anthropic team are always available to provide support and answ

# 使用自定义函数（Recommended）

In [5]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain
    
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, to use Anthropic, you can start by exploring the various products and services they offer. Anthropic is a leading artificial intelligence research company that develops advanced language models and other AI technologies. \n\nSome key ways to engage with Anthropic include:\n\n1. Accessing their public demos and tools, such as the Anthropic Playground, which allows you to interact with and experiment with their language models.\n\n2. Exploring their research publications and blog posts to stay up-to-date on the latest developments in AI and Anthropic's work.\n\n3. Considering Anthropic's API and cloud services, which provide access to their AI models and capabilities for developers and businesses.\n\n4. Participating in Anthropic's community events, workshops, or educational initiatives to learn more about their work and how to effectively leverage their technologies.\n\n5. Exploring potential collaboration or partnership opportunities if you h

# 通过语义相似性路由 - Routing by semantic similarity

In [6]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

# 定义一个物理学教授机器人
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

# 定义一个数学教授机器人
math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# 模板路由
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | chat_anthropic
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
As a physics professor, I would be happy to explain what a black hole is in a concise and easy-to-understand manner.

A black hole is an extremely dense and massive object in space that has such strong gravitational pull that nothing, not even light, can escape from it. It is the result of the gravitational collapse of a massive star at the end of its life cycle.

When a large star runs out of fuel, its core can no longer support itself against its own gravity, and it begins to collapse inward. If the star is massive enough, this collapse continues until the matter is compressed into an infinitely small point called a singularity. The gravitational force around this singularity becomes so strong that it creates an event horizon, which is the point of no return - anything that crosses this boundary, including light, can never escape.

The key features of a black hole are:

1. Singularity - the infinitely dense and small point at the center of the black hole.
2. Event horiz

In [7]:
print(chain.invoke("What's a path integral"))

Using MATH
A path integral is a concept in quantum mechanics and field theory that provides a formulation for the quantum mechanical propagator or transition amplitude between two states. It was developed by the physicist Richard Feynman as an alternative to the standard Hamiltonian formulation of quantum mechanics.

The key ideas behind the path integral formulation are:

1. Superposition of Histories:
   - In quantum mechanics, a particle does not follow a single classical trajectory, but rather explores all possible paths between the initial and final states.
   - The path integral formulation considers the contribution of all possible paths the particle can take, with each path weighted by a complex-valued amplitude.

2. Propagator:
   - The propagator, or transition amplitude, represents the probability amplitude for a particle to transition from one state to another.
   - In the path integral formulation, the propagator is expressed as a sum (or integral) over all possible paths 

# 使用`@chain`装饰器创建Runnables

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

# 创建两个 PROMPT
prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")

@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = chat_anthropic.invoke(prompt_val1)

    # Prompt1 的输出作为 Prompt2 的输入
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | chat_anthropic | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})

custom_chain.invoke("bears")

'The subject of this joke is bears and their feet.\n\nThe punchline plays on the similarity between the words "bear feet" and "bare feet". The joke suggests that bears don\'t wear socks because their feet are naturally "bear feet" - i.e. they are already bare/without socks.\n\nThe humor comes from the wordplay and the absurdity of the idea of bears wearing socks in the first place. The joke relies on the audience making the connection between "bear feet" and "bare feet" to understand the punchline.'

# 管理`Prompt Size`

In [11]:
import os
from operator import itemgetter

from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

os.environ['http_proxy'] = 'http://127.0.0.1:10801'
os.environ['https_proxy'] = 'http://127.0.0.1:10801'

wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=10_000)
)
tools = [wiki]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(temperature=0, api_key=API_KEY, model_name="gpt-3.5-turbo", base_url=BASE_URL)

# 定义一个函数删除历史记录中最早的AI、函数消息对
def condense_prompt(prompt: ChatPromptValue) -> ChatPromptValue:
    messages = prompt.to_messages()
    num_tokens = llm.get_num_tokens_from_messages(messages)
    ai_function_messages = messages[2:]
    while num_tokens > 4_000:
        ai_function_messages = ai_function_messages[2:]
        num_tokens = llm.get_num_tokens_from_messages(
            # 前两条消息是系统和用户消息，所以要加上
            messages[:2] + ai_function_messages
        )
    messages = messages[:2] + ai_function_messages
    return ChatPromptValue(messages=messages)


# 创建一个 Agent，查询维基百科
agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | condense_prompt
    | llm.bind_functions(tools)
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `List of presidents of the United States`


Page: List of presidents of the United States
Summary: The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. Since the office was established in 1789, 45 men have served in 46 presidencies. The first president, George Washington, won a unanimous vote of the Electoral College. Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the number of presidencies and the number of individuals who have served as president.The presidency of William Henry Harrison, who died 31 days after taking office in 1841, was the shortest in Am

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null. (request id: 2024041509284568772389468732640)", 'type': 'invalid_request_error', 'param': 'messages.[2].content', 'code': None}}

# 多链`Multiple Chain`

In [13]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

llm = ChatOpenAI(api_key=API_KEY, model_name="gpt-3.5-turbo", base_url=BASE_URL)

# 创建 chain1，回答 prompt1 的问题
chain1 = prompt1 | llm | StrOutputParser()

# 创建 chain2，回答 prompt2 的问题，依赖于 chain1 的结果
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "chinese"})

'芝加哥市是在美国伊利诺伊州的。'

In [15]:
from langchain_core.runnables import RunnablePassthrough

# prompt1: 生成一种颜色
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
# prompt2: 给定一种颜色，问一种水果
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
# prompt3：给定一种颜色，问一种国旗和国家
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
# prompt4: 问水果和国旗的颜色
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = llm | StrOutputParser()

# 基于用户的提示，使用 prompt1 生成颜色
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
# 使用 prompt2，生成水果
color_to_fruit = prompt2 | model_parser
# 使用 prompt2，生成国旗对应的国家名称
color_to_country = prompt3 | model_parser
# 将各个环节串联起来：颜色 -> 水果/国家 -> 询问颜色
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

prompt = question_generator.invoke("warm")
llm.invoke(prompt)

AIMessage(content='The color of a mango is typically a shade of yellow or orange. The flag of Cyprus consists of a white background with a copper-orange map of the island in the center.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 19, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-24ed38dd-d617-4ed5-919e-f5ae2be782de-0')

# 分支与合并

```
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
      Combine
```

In [16]:
llm = ChatOpenAI(api_key=API_KEY, model_name="gpt-3.5-turbo", base_url=BASE_URL)

# 基于用户的提示，生成参数
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | llm
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 列出正向的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | llm
    | StrOutputParser()
)
# 列出负向的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | llm
    | StrOutputParser()
)
# 最终答复
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | llm
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

chain.invoke({"input": "scrum"})

"While scrum offers numerous benefits for project management, it's essential to acknowledge the potential drawbacks associated with its use. Some challenges include the lack of structure, time-consuming meetings, difficulty in scaling, dependency on team dynamics, and lack of predictability. These issues can impact the effectiveness of the framework if not managed properly.\n\nTo address these challenges, teams can consider establishing clearer guidelines and structures within the scrum framework, optimizing meeting efficiency, seeking additional support for scaling efforts, fostering strong team dynamics through effective communication, and setting realistic expectations around project timelines and deliverables.\n\nBy being mindful of these potential drawbacks and implementing strategies to mitigate them, teams can maximize the benefits of scrum while overcoming its limitations, ultimately leading to successful project outcomes."